<a href="https://colab.research.google.com/github/KacperKaszuba0608/Projects-SQL/blob/main/Analyzing_CIA_Factbook_Data_Using_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyzing CIA Factbook Data Using SQL

In this project I'll work with data from the CIA World Factbook. It is a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:
* `population` — the global population.
* `population_growth` — the annual population growth rate, as a percentage.
* `area` — the total land and water area.

At first we connect with SQLite `factbook.db` databse with the commands below.

In [4]:
!pip install SQLAlchemy==1.4.4 -q

In [5]:
%%capture
!wget -Oq factbook.db http://bioinf-mw.bihz.upwr.edu.pl/resources/dataquest/factbook.db
%load_ext sql
%sql sqlite:///factbook.db

## Overview of the Data

Here are the descriptions for some of the columns:

* `name` — the name of the country.
* `area` — the country's total area (both land and water).
* `area_land` — the country's land area in square kilometers.
* `area_water` — the country's water area in square kilometers.
* `population` — the country's population.
* `population_growth` — the country's population growth as a percentage.
* `birth_rate` — the country's birth rate, or the number of births per year per 1,000 people.
* `death_rate` — the country's death rate, or the number of death per year per 1,000 people.

In [6]:
%%sql
SELECT *
    FROM facts
    LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


After look at the content of database, let's calculate some statistics.

In [7]:
%%sql
SELECT MIN(population) as min_pop, 
    MAX(population) as max_pop,
    MIN(population_growth) as min_pop_growth, 
    MAX(population_growth) as max_pop_growth
    FROM facts;

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


As we see above, in the database is a country with population equals 0 and country with population more than 7.2 billion people. 

## Overview Outliers

To zoom in this column let's write some queries. At first we'll find countires with population equal to the minimum population.

In [8]:
%%sql
SELECT *
    FROM facts
    WHERE population = (SELECT MIN(population) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


SQL query shows that only Antarctica has population equals 0.

Second query find countries with population equal to maximum population.

In [9]:
%%sql
SELECT * 
    FROM facts
    WHERE population = (SELECT MAX(population) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


As we see database contain row with population of whole World.

## Exploring Average Population and Area

Now we are going to recompute the summary statistics you found earlier while excluding the row for the whole world.

In [10]:
%%sql
SELECT
    MIN(population) as min_pop,
    MAX(population) as max_pop,
    MIN(population_growth) as min_pop_growth,
    MAX(population_growth) as max_pop_growth
    FROM facts
    WHERE name != 'World';

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


In [11]:
%%sql
SELECT
    ROUND(AVG(population), 2) as avg_pop,
    ROUND(AVG(area), 2) as avg_area
    FROM facts
    WHERE name != 'World'

 * sqlite:///factbook.db
Done.


avg_pop,avg_area
32242666.57,555093.55


As we see the average of population is 32 million people and 555 thousand square kilometers.

## Finding Densely Populated Countries

We'll build on the query we wrote for the previous to find countries that are densely populated. We'll identify countries that have the following:

* Above-average values for population.
* Below-average values for area.

In [12]:
%%sql
SELECT * 
    FROM facts
    WHERE population > (SELECT AVG(population) FROM facts) AND
          area < (SELECT AVG(area) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3


As we see countries which are densley populated are:
* Bangladesh
* Germany
* Japan
* Philippines
* Thailand
* United Kingdom
* Vietnam